In [1]:
from myonset import Events

## One simple Events object

In this object, 5 events are stored, they occured at time latencies 1s, 2s, 3s, 4s and 5s. They all have the same code, (``1``), and are not associated with a particular channel (``chan = -1``, our convention for events not associated with a channel). 

In [19]:
simple_evts = Events(time=[1,2,3,4,5], code=[1]*5, chan=[-1]*5, sf=1024)

Event latencies are stored in ``lat`` object, in which latencies are stores in both time (in s) and samples. 

In [21]:
print(simple_evts.lat)
print('signal sampling frequency: {}'.format(simple_evts.lat.sf))
print('events samples: {}'.format(simple_evts.lat.sample))
print('events times(s): {}'.format(simple_evts.lat.time))

class Latency, 5 events, sf = 1024.0, sample: [1024 2048 3072 4096 5120], time: [1 2 3 4 5]
signal sampling frequency: 1024.0
events samples: [1024 2048 3072 4096 5120]
events times(s): [1 2 3 4 5]


## One real Events object

Example of one Events object containing two trials (stimulus code is ``4``, response codes are ``16`` and ``32``). 

In [4]:
exp_evts = Events(time=[1,1.5,1.623,1.678,3.5,3.709,3.785], code=[2,4,'onset',16,4,'onset',32],
                  chan=[-1,-1,0,-1,-1,1,-1], sf=1024)
print(exp_evts)

class Events, 7 events, sf = 1024.0 
 latency: class Latency, 7 events, sf = 1024.0, sample: [1024 1536 1662 1718 3584 3798 3876], time: [1.    1.5   1.623 1.678 3.5   3.709 3.785] 
 code: ['2' '4' 'onset' '16' '4' 'onset' '32'] 
 chan: [-1 -1  0 -1 -1  1 -1]


EMG onsets are coded ``onset`` and are associated with a specific channel: channel ``0`` for first onset and channel ``1`` for second onset. To access those particular events:

In [5]:
print(exp_evts.get_events(2))
print(exp_evts.get_events(5))

class Events, 1 events, sf = 1024.0 
 latency: class Latency, 1 events, sf = 1024.0, sample: [1662], time: [1.623] 
 code: ['onset'] 
 chan: [0]
class Events, 1 events, sf = 1024.0 
 latency: class Latency, 1 events, sf = 1024.0, sample: [3798], time: [3.709] 
 code: ['onset'] 
 chan: [1]


To find and get all events with a particular code, or associated with a particular channel, use:

In [6]:
exp_evts.find_and_get_events(code=['2','4'])

3 event(s) selected.


class Events, 3 events, sf = 1024.0 
 latency: class Latency, 3 events, sf = 1024.0, sample: [1024 1536 3584], time: [1.  1.5 3.5] 
 code: ['2' '4' '4'] 
 chan: [-1 -1 -1]

Note that, when specifying several values for one attribute (for instance here several codes), events whose attribute is equal to either value is selected. If several attributes are filled however, only events satisfying all requirements are selected:

In [7]:
exp_evts.find_and_get_events(code=['2','4'],chan=0)

Event(s) not found, nothing was selected.


class Events, 0 events, sf = 1024.0 
 latency: class Latency, 0 events, sf = 1024.0, sample: [], time: [] 
 code: [] 
 chan: []

## Epoch events

The ``EpochEvents`` object is used to store segmented events (or epoch events). ``EpochEvents`` can be obtained by epoching ``Events`` objects, based on the position of reference events for instance. The code of the reference event(s) is defined by the parameter ``code_t0``.

In [8]:
epochs_evts = exp_evts.segment(code_t0=['4'], tmin=-0.5, tmax = 1)
print(epochs_evts)

Found 2 epoch(s).
class EpochEvents, 2 trials, 7 events, sf = 1024.0 


The attribute ``list_evts_trials`` contains the list of ``Events`` object corresponding to each trial. For instance, for trial 0:

In [9]:
epochs_evts.list_evts_trials[0]

class Events, 4 events, sf = 1024.0 
 latency: class Latency, 4 events, sf = 1024.0, sample: [  0 512 638 694], time: [-0.5    0.     0.123  0.178] 
 code: ['2' '4' 'onset' '16'] 
 chan: [-1 -1  0 -1]

The attributes ``tmin``, ``t0`` and ``tmax`` contain ``Latency`` objects storing the starting, time 0 and ending latency of each trial:

In [10]:
epochs_evts.t0

class Latency, 2 events, sf = 1024.0, sample: [1536 3584], time: [1.5 3.5]

The method ``as_continuous`` allows to switch back to continuous events from the epoch events:

In [11]:
epochs_evts.as_continuous()

Checking for duplicates in events...
0 event(s) removed.


(class Events, 7 events, sf = 1024.0 
  latency: class Latency, 7 events, sf = 1024.0, sample: [1024 1536 1662 1718 3584 3798 3876], time: [1.    1.5   1.623 1.678 3.5   3.709 3.785] 
  code: ['2' '4' 'onset' '16' '4' 'onset' '32'] 
  chan: [-1 -1  0 -1 -1  1 -1],
 array([0, 0, 0, 0, 1, 1, 1]))

Note that events outside the epochs boundaries are lost: if epochs are defined from 0 to 0.15s around stimulus, all events not falling in this interval will not be recovered using the ``as_continuous`` method:

In [12]:
short_epochs = exp_evts.segment(code_t0=['4'], tmin=0, tmax = 0.15)
print(short_epochs.list_evts_trials[0])

Found 2 epoch(s).
class Events, 2 events, sf = 1024.0 
 latency: class Latency, 2 events, sf = 1024.0, sample: [  0 126], time: [0.    0.123] 
 code: ['4' 'onset'] 
 chan: [-1  0]


In [13]:
continuous_evts = short_epochs.as_continuous()[0]
print(continuous_evts)

Checking for duplicates in events...
0 event(s) removed.
class Events, 3 events, sf = 1024.0 
 latency: class Latency, 3 events, sf = 1024.0, sample: [1536 1662 3584], time: [1.5   1.623 3.5  ] 
 code: ['4' 'onset' '4'] 
 chan: [-1  0 -1]


To avoid losing events, one good practice is to always combine original continuous events with the events resulting from ``as_continuous`` method. By setting the parameter ``drop_duplic`` to True, duplicated events will be removed automatically. 

In [14]:
continuous_evts.add_events(exp_evts, drop_duplic=True)
print(continuous_evts)

Checking for duplicates in events...
3 event(s) removed.
class Events, 7 events, sf = 1024.0 
 latency: class Latency, 7 events, sf = 1024.0, sample: [1024 1536 1662 1718 3584 3798 3876], time: [1.    1.5   1.623 1.678 3.5   3.709 3.785] 
 code: ['2' '4' 'onset' '16' '4' 'onset' '32'] 
 chan: [-1 -1  0 -1 -1  1 -1]
